In [1]:
 # Suppresses warnings for cleaner output.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

# Load datasets from CSV files.
recipes = pd.read_csv("recipes.csv")
ratings = pd.read_csv("interactions.csv")

In [3]:
# Print columns of both datasets for inspection.
print(recipes.columns)
print(ratings.columns)

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')
Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')


In [4]:
# Checking the ratings DataFrame.
ratings

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [5]:
# Count how many ratings are zero and display the count.
count_zero_ratings = ratings[ratings['rating'] == 0].count()
count_zero_ratings

user_id      60847
recipe_id    60847
date         60847
rating       60847
review       60847
dtype: int64

In [6]:
# Remove entries where ratings are zero as they might not be useful for recommendation.
ratings.drop(ratings[ratings['rating'] == 0].index, inplace=True)

In [7]:
# Check again to ensure no zero ratings exist.
count_zero_ratings = ratings[ratings['rating'] == 0].count()
count_zero_ratings

user_id      0
recipe_id    0
date         0
rating       0
review       0
dtype: int64

In [8]:
# Check for and display any missing values across columns.
print(recipes.isnull().sum())
print(ratings.isnull().sum())

name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64
user_id        0
recipe_id      0
date           0
rating         0
review       169
dtype: int64


In [9]:
# Drop rows with any missing values in the DataFrames.
# The column description won't be relevant so we won't be dropping those roles.
ratings.dropna(inplace=True)
recipes.dropna(subset=['name'], inplace=True)

In [10]:
# Create new DataFrames with essential columns for further analysis.
recipes_df = recipes[['name', 'id', 'ingredients']]
ratings_df = ratings[['user_id', 'recipe_id', 'rating']]

In [11]:
# Rename 'id' column to 'recipe_id' to maintain consistency across DataFrames
recipes_df.rename(columns={'id': 'recipe_id'}, inplace=True)

In [12]:
# Save the new datasets to new CSV files
recipes_df.to_csv('recipes_df.csv', index=False)
ratings_df.to_csv('ratings_df.csv', index=False)

In [13]:
# Print columns of the modified recipes DataFrame to confirm changes.
print(recipes_df.columns)
print(ratings_df.columns)

Index(['name', 'recipe_id', 'ingredients'], dtype='object')
Index(['user_id', 'recipe_id', 'rating'], dtype='object')


In [14]:
# Check for duplicates
duplicates = recipes_df[recipes_df.duplicated()]
print(duplicates)
duplicates_ratings = ratings_df[ratings_df.duplicated()]
print(duplicates_ratings)

Empty DataFrame
Columns: [name, recipe_id, ingredients]
Index: []
Empty DataFrame
Columns: [user_id, recipe_id, rating]
Index: []


In [15]:
# Calculate the average rating for each recipe
average_ratings = ratings_df.groupby('recipe_id')['rating'].mean().reset_index()
average_ratings.columns = ['recipe_id', 'average_rating']

# Merge the average ratings with the recipes DataFrame
recipes_df = pd.merge(recipes_df, average_ratings, on='recipe_id', how='left')

In [16]:
recipes_df

,name,recipe_id,ingredients,average_rating
0,arriba baked winter squash mexican style,137739,"['winter squash', 'mexican seasoning', 'mixed ...",5.000000
1,a bit different breakfast pizza,31490,"['prepared pizza crust', 'sausage patty', 'egg...",4.666667
2,all in the kitchen chili,112140,"['ground beef', 'yellow onions', 'diced tomato...",4.000000
3,alouette potatoes,59389,"['spreadable cheese with garlic and herbs', 'n...",4.500000
4,amish tomato ketchup for canning,44061,"['tomato juice', 'apple cider vinegar', 'sugar...",5.000000
...,...,...,...,...
231631,zydeco soup,486161,"['celery', 'onion', 'green sweet pepper', 'gar...",5.000000
231632,zydeco spice mix,493372,"['paprika', 'salt', 'garlic powder', 'onion po...",5.000000
231633,zydeco ya ya deviled eggs,308080,"['hard-cooked eggs', 'mayonnaise', 'dijon must...",5.000000
231634,cookies by design cookies on a stick,298512,"['butter', 'eagle brand condensed milk', 'ligh...",1.000000
